In [1]:
from __future__ import print_function, division
import os, random, time, copy
import numpy as np
import pandas as pd
import os.path as path
import scipy.io as sio
from scipy import misc
from scipy import ndimage, signal
import scipy
import pickle
import sys
import math
import matplotlib.pyplot as plt
from io import BytesIO


import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler 
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import torchvision.utils as vutils


import warnings 
warnings.filterwarnings("ignore")
print(sys.version)
print(torch.__version__)


manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)


import faiss
import os
import pandas as pd
import seaborn as sn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import faiss
from IPython import display
from torch.optim.lr_scheduler import OneCycleLR


3.6.13 |Anaconda, Inc.| (default, Feb 23 2021, 21:15:04) 
[GCC 7.3.0]
1.8.0+cu111
Random Seed:  999


In [2]:


tarad=torch.load('./savedata/tarad1225.pkl')



featesttrain=torch.load('./savedata/featesttrain1229.pkl') 

featestA=torch.load('./savedata/featestAzyori0106.pkl')

featestD=torch.load('./savedata/featestDzy0106.pkl')



featesttrain2=torch.load('./savedata/featesttrain0102-noadv.pkl')

featestA2=torch.load('./savedata/featestA0102-noadv.pkl')

featestD2=torch.load('./savedata/featestD0102-noadv.pkl')



In [3]:
dd=[]
newtrain=[]
for i in range(len(featesttrain)):
    dd=[]
    dd.append(featesttrain[i].detach().cpu().numpy())
    dd=np.concatenate(dd)
    newtrain.append(dd)
    
    
dd=[]
newtestA=[]
for i in range(len(featestA)):
    dd=[]
    dd.append(featestA[i].detach().cpu().numpy())
    dd=np.concatenate(dd)
    newtestA.append(dd)
    
dd=[]
newtestD=[]
for i in range(len(featestD)):
    dd=[]
    dd.append(featestD[i].detach().cpu().numpy())
    dd=np.concatenate(dd)
    newtestD.append(dd)

    
train_features = np.array(torch.tensor(newtrain))
test_features = np.array(torch.tensor(newtestA))
ood_features = np.array(torch.tensor(newtestD))

In [4]:
dd=[]
newtrain2=[]
for i in range(len(featesttrain2)):
    dd=[]
    dd.append(featesttrain2[i].detach().cpu().numpy())
    dd=np.concatenate(dd)
    newtrain2.append(dd)
    
    
dd=[]
newtestA2=[]
for i in range(len(featestA2)):
    dd=[]
    dd.append(featestA2[i].detach().cpu().numpy())
    dd=np.concatenate(dd)
    newtestA2.append(dd)
    
dd=[]
newtestD2=[]
for i in range(len(featestD2)):
    dd=[]
    dd.append(featestD2[i].detach().cpu().numpy())
    dd=np.concatenate(dd)
    newtestD2.append(dd)
  

train_features2 = np.array(torch.tensor(newtrain2))
test_features2 = np.array(torch.tensor(newtestA2))
ood_features2 = np.array(torch.tensor(newtestD2))

In [5]:

train_features00 = np.concatenate([train_features,train_features2],axis=1)
test_features00 = np.concatenate([test_features,test_features2],axis=1)
ood_features00 = np.concatenate([ood_features,ood_features2],axis=1)

In [6]:
index = faiss.IndexFlatL2(train_features00.shape[1])
index.add(train_features00)

In [7]:
k = 1
in_l2_distances, in_k_closest_points = index.search(test_features00, k)

In [8]:

in_scores = in_l2_distances[:, -1]

In [9]:

out_l2_distances, out_k_closest_points = index.search(ood_features00, k)

In [10]:
out_scores = out_l2_distances[:, -1]

In [11]:
in_scores.sort()
out_scores.sort()

In [12]:
threshold= 0.007583484#Thresholds can be selected from the validation set


In [13]:
print("threshold:",threshold)

tp = 0
tn = 0
fp = 0
fn = 0
testAres=[]
testDres=[]
for point in in_scores:
    if point <= threshold:
        testAres.append(0)
        tp += 1
    else:
        testAres.append(1)
        fn += 1

for point in out_scores:
    if point <= threshold:
        testDres.append(0)
        fp += 1
    else:
        testDres.append(1)
        tn += 1
        
tp_rate = tp / (tp + fn)
tn_rate = tn / (tn + fp)
fp_rate = fp / (fp + tn)
fn_rate = fn / (fn + tp)

result = {
    "tp": tp,
    "tn": tn,
    "fp": fp,
    "fn": fn,
    "tpr": tp_rate,
    "fpr": fp_rate,
}

print(result)

threshold: 0.007583484
{'tp': 2184, 'tn': 233, 'fp': 119, 'fn': 40, 'tpr': 0.9820143884892086, 'fpr': 0.3380681818181818}


In [14]:
testres=testAres+testDres

In [15]:
y_test=[]

for i in range(len(tarad)):
    if tarad[i]<5:
        y_test.append(0)
    else:
        y_test.append(1)

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print("Accuracy of DeepkNN in localizing unknown attacks without adv:",accuracy_score(y_test, testres))


Accuracy of DeepkNN in localizing unknown attacks without adv: 0.9382763975155279


In [17]:
torch.save(testres,'./savedata/testres1231-noadv.pkl')